In [ ]:
# Compute KL-divergence between predictive distributions at trial t and t+1
def KLdiv(prob_from,prob_to):
    klv = np.array([sum(prob_to[i] * np.log(prob_to[i]/prob_from[i])) for i in range(len(prob_from))])
    return klv

# Compute Surprise between predictive distribution and likelihood at trial t
def Surprise(lik,pre):
    s = np.array([-np.log(sum(lik[i]*pre[i])) for i in range(len(pre))])
    return s

# Compute the entropy of the predictive distribution at trial t
def Entropy(pre):
    h = np.array([-sum(pre[i] * np.log(pre[i])) for i in range(len(pre))])
    return h

def ITmeasures(D,M,params):
    # D: observed data array [Z, Y]
    # M: Empirical Matching Matrix
    # params: parameter vector [epsilon, xi]
    epsilon_par, xi_par = params
    vartheta_par=.99
    lambda_par=.99
    z = D[:,0]
    y = D[:,1]
    max_trials = len(M)
    s = np.zeros((max_trials,3), dtype=int) # Initialize the signal vector
    omega = np.zeros((max_trials,3)) # Initialize the vector of the attention to reward process
    pre = np.zeros((max_trials,3)) # Initialize the vector of the predictive (prior) probabilities
    lik = np.zeros((max_trials,3)) # Initialize the vector of likelihoods
    post = np.zeros((max_trials,3)) # Initialize the vector of posteriors
    pre[0] = np.array([1/3,1/3,1/3])
    s[0] = y[0] == M[0] 
    omega[0] = smoothVec(z[0],s[0],epsilon_par,lambda_par,xi_par,np.array([0.5,0.5,0.5]))
    lik[0] = Likelihood(s[0],z[0],vartheta_par)
    post[0] = Posterior(lik[0],pre[0])
    G = TransMat(omega[0])
    for t in range(1,max_trials):
        pre[t] = ChapmanKolmogorov(G, post[t-1]) # Compute predictive distribution
        s[t] = y[t] == M[t] # signal
        omega[t] = smoothVec(z[t],s[t],epsilon_par,lambda_par,xi_par,omega[t-1]) # Update omega
        lik[t] = Likelihood(s[t],z[t],vartheta_par) # Compute Likelihood of the observation given the states
        post[t] = Posterior(lik[t],pre[t]) # Compute posterior distribution over the hidden states
        G = TransMat(omega[t]) # Update Stability Matrix
    
    B = KLdiv(pre[0:127],pre[1:128])
    I = Surprise(lik[0:127],pre[0:127])
    H = Entropy(pre[0:127])
    # KL-divergence, Surprise, Entropy 
    return np.c_[B,I,H]